In [1]:
import numpy as np

class GeneStruct:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.seq = ""
        

class FastaGeneReader:
    def __init__(self):
        self.gene_list = None
    
    def read_from_file(self, file):
        self.gene_list = []
        
        with open(file) as fin:
            for line in fin.readlines():
                if line[0] == '>':
                    blank_idx = line.find(' ')
                    
                    gene = GeneStruct(
                        line[1:blank_idx],
                        line[blank_idx+1:].strip()
                    )
                    self.gene_list.append(gene)
                else:
                    self.gene_list[-1].seq += line.strip()

In [2]:
class NwAlgo:
    def __init__(self, seq1, seq2, match=2, mismatch=-1, gap=-2):
        self.seq1 = seq1
        self.seq2 = seq2
        self.match = match
        self.mismatch = mismatch
        self.gap = gap
        
        self.l1 = len(seq1)
        self.l2 = len(seq2)
        
        self.mat_score = np.zeros([self.l1+1, self.l2+1])
        self.mat_arrow = np.zeros([self.l1+1, self.l2+1], dtype=int)
        
        self.best_path_list = []
        
        self.stack = []

        # self.mat_best_path = ([None] * self.l2) * self.l1
        
        
    def sub_problem(self, i, j):
        if i == 0 and j == 0:
            return
        
        seq1 = self.seq1
        seq2 = self.seq2

        if i > 0 and j >0:
            score2_base = self.mat_score[i-1][j-1]

            if seq1[i-1] == seq2[j-1]:
                score2 = self.match + score2_base
            else:
                score2 = self.mismatch + score2_base
        else:
            score2 = -np.inf

        if i > 0:
            score3 = self.mat_score[i-1][j] + self.gap
        else:
            score3 = -np.inf

        if j > 0:
            score4 = self.mat_score[i][j-1] + self.gap
        else:
            score4 = -np.inf

        score = max(score2, score3, score4)
        self.mat_score[i][j] = score
        if score2 == score:
            self.mat_arrow[i][j] += 1
        if score3 == score:
            self.mat_arrow[i][j] += 2
        if score4 == score:
            self.mat_arrow[i][j] += 4
     
    
    def best_path_step(self, path_now, i, j):
        # print(i, j)
        
        if i==0 and j==0:
            # print("num_sols:", len(self.best_path_list))
            # print("num_stack:", len(self.stack))
            self.best_path_list.append(path_now)
        
        if self.mat_arrow[i][j] & 1:
            self.stack.append((path_now + [(i-1, j-1)], i-1, j-1))
            # self.best_path_step(path_now + [(i-1, j-1)], i-1, j-1)
        
        if self.mat_arrow[i][j] & 2:
            self.stack.append((path_now + [(i-1, j)], i-1, j))
            # self.best_path_step(path_now + [(i-1, j)], i-1, j)
            
        if self.mat_arrow[i][j] & 4:
            self.stack.append((path_now + [(i, j-1)], i, j-1))
            # self.best_path_step(path_now + [(i, j-1)], i, j-1)
            
            
    def find_best_paths(self, maximum_solutions=10):
        self.stack = []
        self.stack.append(([(self.l1, self.l2)], self.l1, self.l2))
        while(len(self.stack) != 0):
            if len(self.best_path_list) >= maximum_solutions:
                return [self.decode_path(path) for path in self.best_path_list]
            
            path_now, i, j = self.stack.pop(-1)
            self.best_path_step(path_now, i, j) 
            
    def solve(self):
        for i in range(self.l1+1):
            for j in range(self.l2+1):
                self.sub_problem(i, j)

        # self.best_path_step([(self.l1, self.l2)], self.l1, self.l2)
        
        
        # best_alignments = [self.decode_path(path) for path in self.best_path_list]
        
        
        return self.mat_score[self.l1-1][self.l2-1]#, best_alignments
    
            
    def decode_path(self, path, size_per_line=80):
        last_point = None
        
        seq1 = []
        seq2 = []
        
        for point in path[::-1]:
            if last_point is not None:
                if last_point[0] == point[0]:
                    seq1.append('-')
                else:
                    seq1.append(self.seq1[last_point[0]])
                    
                if last_point[1] == point[1]:
                    seq2.append('-')
                else:
                    seq2.append(self.seq2[last_point[1]])
                
            last_point = point
            
            
        output_str = []
        for k in range(0, len(seq1), size_per_line):
            output_str.append(''.join(seq1[k: k+size_per_line])+'\n')
            output_str.append('|'*len(seq1[k: k+size_per_line]) + '\n')
            output_str.append(''.join(seq2[k: k+size_per_line])+'\n')
            output_str.append('\n')
            
        # return ''.join(seq1), ''.join(seq2)
        
        return ''.join(output_str)
    
    
    def save_output(self, filename):
        np.savetxt(filename, self.mat_score, fmt='%.5e')
        
        with open(filename, 'a') as f:
            f.write('\n')
            # f.write(np.array_str(solver.mat_score)+'\n')
            for i, path in enumerate(self.best_path_list):
                f.write("path_" + str(i) + '>' + '\n' + self.decode_path(path))
            f.write("best_score:" + str(self.mat_score[self.l1][self.l2]) + '\n')

# homogogous_genes

In [3]:
reader1 = FastaGeneReader()
reader1.read_from_file("input/homo1.fa")

reader2 = FastaGeneReader()
reader2.read_from_file("input/homo2.fa")

# match=2, mismatch=-1, gap=0
## find 5 optimal paths

In [4]:
solver = NwAlgo(reader1.gene_list[0].seq, reader2.gene_list[0].seq, 2, -1, 0)
score = solver.solve()
result = solver.find_best_paths(5)
solver.save_output("output/homo_result1.txt")

# match=2, mismatch=-1, gap=-2.5
## find 10 optimal paths

In [5]:
solver = NwAlgo(reader1.gene_list[0].seq, reader2.gene_list[0].seq, 2, -1, -2.5)
score = solver.solve()
result = solver.find_best_paths(10)
solver.save_output("output/homo_result2.txt")

# human and hamster insulin

In [6]:
reader1 = FastaGeneReader()
reader1.read_from_file("input/human_insulin.fa")

reader2 = FastaGeneReader()
reader2.read_from_file("input/hamster_insulin.fa")

# match=2, mismatch=-1, gap=0
## find 5 optimal paths

In [7]:
solver = NwAlgo(reader1.gene_list[0].seq, reader2.gene_list[0].seq, 2, -1, 0)
score = solver.solve()
result = solver.find_best_paths(5)
solver.save_output("output/insulin_result1.txt")

# match=2, mismatch=-1, gap=-2.5
## find 10 optimal paths

In [8]:
solver = NwAlgo(reader1.gene_list[0].seq, reader2.gene_list[0].seq, 2, -1, -2.5)
score = solver.solve()
result = solver.find_best_paths(10)
solver.save_output("output/insulin_result2.txt")